In [3]:
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, GRU, LSTM, Bidirectional, Dropout, Flatten, ReLU, BatchNormalization
from keras.optimizers import Adam, Lion
from keras.callbacks import EarlyStopping

from keras.utils import custom_object_scope

In [5]:
max_features = 10000
maxlen = 500

In [6]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 [==============================] - 0s 0us/step


Передпроцесинг

In [7]:
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

###Simple RNN

In [11]:
model_rnn1 = Sequential()
model_rnn1.add(Embedding(max_features, 32))
model_rnn1.add(SimpleRNN(128, return_sequences=True))
model_rnn1.add(Dropout(0.2))
model_rnn1.add(SimpleRNN(64, return_sequences=False))
model_rnn1.add(Dropout(0.2))
model_rnn1.add(Dense(1, activation='sigmoid'))

model_rnn1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_rnn1.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 128)         20608     
                                                                 
 dropout_1 (Dropout)         (None, None, 128)         0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 353025 (1.35 MB)
Trainable params: 35302

In [7]:
model_rnn1.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/5
196/196 [==============================] - 249s 1s/step - loss: 0.7354 - accuracy: 0.5027 - val_loss: 0.6909 - val_accuracy: 0.5242
Epoch 2/5
196/196 [==============================] - 273s 1s/step - loss: 0.7122 - accuracy: 0.5212 - val_loss: 1.0245 - val_accuracy: 0.5026
Epoch 3/5
196/196 [==============================] - 244s 1s/step - loss: 0.6051 - accuracy: 0.6798 - val_loss: 0.5292 - val_accuracy: 0.7383
Epoch 4/5
196/196 [==============================] - 281s 1s/step - loss: 0.5049 - accuracy: 0.7641 - val_loss: 0.4898 - val_accuracy: 0.7776
Epoch 5/5
196/196 [==============================] - 277s 1s/step - loss: 0.4563 - accuracy: 0.7926 - val_loss: 0.4180 - val_accuracy: 0.8126


In [8]:
test_loss, test_acc = model_rnn1.evaluate(x_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc * 100))

782/782 [==============================] - 65s 83ms/step - loss: 0.4180 - accuracy: 0.8126
Test Loss: 0.4179951846599579
Test Accuracy: 81.25600218772888


In [14]:
model_rnn_3 = Sequential()
model_rnn_3.add(Embedding(max_features, 64))
model_rnn_3.add(SimpleRNN(128, return_sequences=True))
model_rnn_3.add(Dropout(0.5))
model_rnn_3.add(SimpleRNN(64, return_sequences=False))
model_rnn_3.add(Dropout(0.5))

model_rnn_3.add(Dense(1, activation='sigmoid'))

with custom_object_scope({'Lion': Lion}):
  model_rnn_3.compile(optimizer='Lion', loss='binary_crossentropy', metrics=['accuracy'])

# Додайте EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_rnn_3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 64)          640000    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, None, 128)         24704     
                                                                 
 dropout_3 (Dropout)         (None, None, 128)         0         
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 677121 (2.58 MB)
Trainable params: 67712

In [14]:
model_rnn_3.fit(x_train, y_train, epochs=10, batch_size=100, validation_data=(x_test, y_test), callbacks=[early_stopping])

test_loss, test_acc = model_rnn_3.evaluate(x_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc * 100))

Epoch 1/10
250/250 [==============================] - 303s 1s/step - loss: 0.2611 - accuracy: 0.9010 - val_loss: 0.3384 - val_accuracy: 0.8627
Epoch 2/10
250/250 [==============================] - 294s 1s/step - loss: 0.1571 - accuracy: 0.9450 - val_loss: 0.4324 - val_accuracy: 0.8416
Epoch 3/10
250/250 [==============================] - 297s 1s/step - loss: 0.0813 - accuracy: 0.9740 - val_loss: 0.5924 - val_accuracy: 0.8422
Epoch 4/10
782/782 [==============================] - 63s 80ms/step - loss: 0.3384 - accuracy: 0.8627
Test Loss: 0.33838602900505066
Test Accuracy: 86.27200126647949


Висновки: найкращою виявилася model_rnn_3 з 2-ма SimpleRNN шарами та оптимізатором Lion і застосуванням early_stopping. Найпростіша RNN-модель без Dropout і з одним шаром показала найгірші результати (72%).

###LSTM model

model_lstm_1 - максимально проста

In [15]:
model_lstm_1 = Sequential()
model_lstm_1.add(Embedding(max_features, 64))
model_lstm_1.add(LSTM(64))
model_lstm_1.add(Dense(1, activation='sigmoid'))

model_lstm_1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_lstm_1.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 64)          640000    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 673089 (2.57 MB)
Trainable params: 673089 (2.57 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model_lstm_1.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))

test_loss, test_acc = model_lstm_1.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc * 100))

Epoch 1/5
196/196 [==============================] - 247s 1s/step - loss: 0.5491 - accuracy: 0.7091 - val_loss: 0.3631 - val_accuracy: 0.8466
Epoch 2/5
196/196 [==============================] - 281s 1s/step - loss: 0.3323 - accuracy: 0.8637 - val_loss: 0.2951 - val_accuracy: 0.8794
Epoch 3/5
196/196 [==============================] - 268s 1s/step - loss: 0.2716 - accuracy: 0.8930 - val_loss: 0.3000 - val_accuracy: 0.8762
Epoch 4/5
196/196 [==============================] - 266s 1s/step - loss: 0.2423 - accuracy: 0.9068 - val_loss: 0.3465 - val_accuracy: 0.8562
Epoch 5/5
196/196 [==============================] - 266s 1s/step - loss: 0.2242 - accuracy: 0.9171 - val_loss: 0.3036 - val_accuracy: 0.8775
Test Loss: 0.3035857379436493
Test Accuracy: 87.74799704551697


model_lstm_2 - 2 LSTM-шари

In [22]:
model_lstm_2 = Sequential()
model_lstm_2.add(Embedding(max_features, 32))
model_lstm_2.add(LSTM(128, return_sequences=True))
model_lstm_2.add(Dropout(0.2))
model_lstm_2.add(LSTM(64))
model_lstm_2.add(Dropout(0.2))

model_lstm_2.add(Dense(1, activation='sigmoid'))

model_lstm_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model_lstm_2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 32)          320000    
                                                                 
 lstm_5 (LSTM)               (None, None, 128)         82432     
                                                                 
 dropout_8 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_6 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 451905 (1.72 MB)
Trainable params: 45190

In [24]:
model_lstm_2.fit(x_train, y_train, epochs=5, batch_size=200, validation_data=(x_test, y_test), callbacks=[early_stopping])

Epoch 1/5
125/125 [==============================] - 672s 5s/step - loss: 0.5019 - accuracy: 0.7538 - val_loss: 0.3630 - val_accuracy: 0.8462
Epoch 2/5
125/125 [==============================] - 676s 5s/step - loss: 0.3233 - accuracy: 0.8713 - val_loss: 0.3416 - val_accuracy: 0.8574
Epoch 3/5
125/125 [==============================] - 682s 5s/step - loss: 0.2336 - accuracy: 0.9120 - val_loss: 0.3276 - val_accuracy: 0.8634
Epoch 4/5
125/125 [==============================] - 658s 5s/step - loss: 0.1860 - accuracy: 0.9336 - val_loss: 0.3569 - val_accuracy: 0.8646
Epoch 5/5
125/125 [==============================] - 688s 6s/step - loss: 0.1660 - accuracy: 0.9414 - val_loss: 0.3548 - val_accuracy: 0.8666


In [25]:
test_loss, test_acc = model_lstm_2.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc * 100))

Test Loss: 0.3547706604003906
Test Accuracy: 86.66399717330933


Висновки: model_lstm_1 з rmsprop оптимізатором та early_stopping виявилася точнішою за model_lstm_2 і навчалася значно швидше. Отже, нема сенсу ускладнювати LSMT-модель додатковими LSTM-шарами.

#Bidirectional

Проста Bidirectional-модель

In [26]:
model_bi_1 = Sequential()
model_bi_1.add(Embedding(max_features, 32))
model_bi_1.add(Bidirectional(LSTM(64)))
model_bi_1.add(Dropout(0.2))

model_bi_1.add(Dense(1, activation='sigmoid'))

model_bi_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_bi_1.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 32)          320000    
                                                                 
 bidirectional (Bidirection  (None, 128)               49664     
 al)                                                             
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 1)                 129       
                                                                 
Total params: 369793 (1.41 MB)
Trainable params: 369793 (1.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
model_bi_1.fit(x_train, y_train, epochs=5, batch_size=256, validation_data=(x_test, y_test), callbacks=[early_stopping])
test_loss, test_acc = model_bi_1.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc * 100))

Epoch 1/5
98/98 [==============================] - 311s 3s/step - loss: 0.5357 - accuracy: 0.7396 - val_loss: 0.3808 - val_accuracy: 0.8537
Epoch 2/5
98/98 [==============================] - 312s 3s/step - loss: 0.2973 - accuracy: 0.8845 - val_loss: 0.3388 - val_accuracy: 0.8591
Epoch 3/5
98/98 [==============================] - 314s 3s/step - loss: 0.2218 - accuracy: 0.9180 - val_loss: 0.3091 - val_accuracy: 0.8704
Epoch 4/5
98/98 [==============================] - 308s 3s/step - loss: 0.1685 - accuracy: 0.9423 - val_loss: 0.3108 - val_accuracy: 0.8774
Epoch 5/5
98/98 [==============================] - 308s 3s/step - loss: 0.1470 - accuracy: 0.9501 - val_loss: 0.3330 - val_accuracy: 0.8711
Test Loss: 0.33300507068634033
Test Accuracy: 87.11199760437012


Bidirectional-модель на 2 шари

In [ ]:
model_bi_2 = Sequential()
model_bi_2.add(Embedding(max_features, 32))
model_bi_2.add(Bidirectional(LSTM(128, return_sequences=True)))
model_bi_2.add(BatchNormalization())
model_bi_2.add(Bidirectional(LSTM(64)))
model_bi_2.add(BatchNormalization())


model_bi_2.add(Dense(1, activation='sigmoid'))

with custom_object_scope({'Lion': Lion}):
  model_bi_2.compile(optimizer='Lion', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_bi_2.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, None, 32)          160000    
                                                                 
 bidirectional_8 (Bidirecti  (None, None, 256)         164864    
 onal)                                                           
                                                                 
 batch_normalization (Batch  (None, None, 256)         1024      
 Normalization)                                                  
                                                                 
 bidirectional_9 (Bidirecti  (None, 128)               164352    
 onal)                                                           
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                    

In [ ]:
model_bi_2.fit(x_train, y_train, epochs=10, batch_size=256, validation_data=(x_test, y_test), callbacks=[early_stopping])

Epoch 1/10
98/98 [==============================] - 536s 5s/step - loss: 0.6343 - accuracy: 0.6304 - val_loss: 0.6887 - val_accuracy: 0.6268
Epoch 2/10
98/98 [==============================] - 525s 5s/step - loss: 0.3607 - accuracy: 0.8382 - val_loss: 0.6126 - val_accuracy: 0.6484
Epoch 3/10
98/98 [==============================] - 526s 5s/step - loss: 0.2348 - accuracy: 0.9047 - val_loss: 0.5156 - val_accuracy: 0.8480
Epoch 4/10
98/98 [==============================] - 523s 5s/step - loss: 0.1725 - accuracy: 0.9325 - val_loss: 0.4032 - val_accuracy: 0.8209
Epoch 5/10
98/98 [==============================] - 521s 5s/step - loss: 0.1170 - accuracy: 0.9568 - val_loss: 0.3863 - val_accuracy: 0.8300
Epoch 6/10
98/98 [==============================] - 520s 5s/step - loss: 0.0747 - accuracy: 0.9741 - val_loss: 0.4997 - val_accuracy: 0.8422
Epoch 7/10
98/98 [==============================] - 522s 5s/step - loss: 0.0479 - accuracy: 0.9846 - val_loss: 0.6526 - val_accuracy: 0.8394
Epoch 8/10
98

In [ ]:
test_loss, test_acc = model_bi_2.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc * 100))

Test Loss: 0.3862709105014801
Test Accuracy: 83.00399780273438


Висновки: model_bi_1 з одним Bidirectional-леєром та оптимізатором Адам виявилася максимально точною та досить швидкою у навчанні. Отже навряд чи є сенс додавати дотакові шари.

DEEP

In [ ]:
model_deep_3 = Sequential()


model_deep_3.add(Embedding(max_features, 32, input_length=maxlen))
model_deep_3.add(Bidirectional(LSTM(64, return_sequences=True)))
model_deep_3.add(Dropout(0.2))
model_deep_3.add(Bidirectional(LSTM(32, return_sequences=True)))
model_deep_3.add(Dropout(0.2))
model_deep_3.add(LSTM(16))
model_deep_3.add(Dropout(0.2))

model_deep_3.add(Dense(1, activation='sigmoid'))

model_deep_3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_deep_3.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 200, 32)           160000    
                                                                 
 bidirectional_6 (Bidirecti  (None, 200, 128)          49664     
 onal)                                                           
                                                                 
 dropout_43 (Dropout)        (None, 200, 128)          0         
                                                                 
 bidirectional_7 (Bidirecti  (None, 200, 64)           41216     
 onal)                                                           
                                                                 
 dropout_44 (Dropout)        (None, 200, 64)           0         
                                                                 
 lstm_38 (LSTM)              (None, 16)              

In [ ]:
model_deep_3.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 412s 772ms/step - loss: 0.4177 - accuracy: 0.8024 - val_loss: 0.3212 - val_accuracy: 0.8643
Epoch 2/10
391/391 [==============================] - 314s 805ms/step - loss: 0.2652 - accuracy: 0.8966 - val_loss: 0.3267 - val_accuracy: 0.8677
Epoch 3/10
391/391 [==============================] - 315s 806ms/step - loss: 0.2189 - accuracy: 0.9177 - val_loss: 0.3270 - val_accuracy: 0.8689
Epoch 4/10
391/391 [==============================] - 314s 803ms/step - loss: 0.1766 - accuracy: 0.9354 - val_loss: 0.3825 - val_accuracy: 0.8646
Epoch 5/10
391/391 [==============================] - 303s 777ms/step - loss: 0.1597 - accuracy: 0.9408 - val_loss: 0.4041 - val_accuracy: 0.8656
Epoch 6/10
391/391 [==============================] - 314s 803ms/step - loss: 0.1276 - accuracy: 0.9540 - val_loss: 0.4160 - val_accuracy: 0.8517
Epoch 7/10
391/391 [==============================] - 315s 805ms/step - loss: 0.1199 - accuracy: 0.9570 - val_loss: 0.4547 -

In [ ]:
test_loss, test_acc = model_deep_3.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc * 100))

Test Loss: 0.5142635703086853
Test Accuracy: 84.87600088119507


Загалом для даного датасету мабуть недоцільно використовувати Deep-моделі, навчання яких триває дуже довго, а результат виходить навіть нижчим ніж в RNN та LSTM-моделях. Найкраще показали себе двошарова RNN-модель з early_stopping та Lion. Дві LSTM-моделі на 1 та 2 шари дали непоагний результат (87 та 86%).
Частково я використовувала deep-моделі в LSTM та Bidirectional, прагнучи отримати кращі показники.